# Préambule

Ce notebook est écrit sous Google Colab.
Colab est un service Google gratuit qui permet de créer et d'exécuter des notebooks Jupyter.
Colab est particulièrement adapté au machine learning, à la science des données et à l'enseignement.

# Introduction

## Rappel
Ce notebook fait suite au TP Perceptron dans lequel nous avions construit un petit réseau de neurones - appelé perceptron - pour effectuer de la reconnaissance de caractères.


<img src="https://github.com/PlantecJY/python-mic/raw/master/principe.png" alt="Drawing" width="300" align="center"/>


Nous avions suivi les étapes ci-dessous :
* Structuration des données pour l’apprentissage (un dictionnaire de chiffres écrits représentés par des tableaux 5*6, comme dans l'exemple ci-dessous

```
nb9b = [[1,1,1,1,1],
        [1,0,0,0,1],
        [1,0,0,0,1],
        [1,1,1,1,1],
        [0,0,0,0,1],
        [1,1,1,1,1]]
```
* Création du réseau de neurones : une couche d'entrée de dimension 300 et une couche de sortie de dimension 10 ; l'information passée à chaque neurone de sortie était simplement la combinaison linéaire des neurones reliés à cette sortie pondérée par le poids de chaque neurone ;
* Test du fonctionnement du réseau de neurones ;
* Apprentissage, c'est-dire-optimisation du poids de chaque neurone ;
* Utilisation du réseau de neurones pour une faire une prédiction : on fournit un chiffre sous forme de tableau et on demande au réseau de prédire ce chiffre.

## Objectif du présent TP
Dans ce notebook, nous allons poursuivre l'exploration des réseaux de neurones et de l'apprentissage supervisé dans le domaine de la reconnaissance de caractères (OCR).
Cette fois, à l'aide de bibliothèques spécialisées, nous allons construire un réseau de neurones (notre "modèle") contenant plusieurs couches de neurones ; puis nous allons compiler ce modèle ; nous allons ensuite entraîner ce modèle/réseau à l'aide de grandes bibliothèques d'images de caractères entre 0 et 9 ; nous allons évaluer la précision de ce modèle sur un ensemble d'images de test ; enfin, nous allons utiliser ce modèle pour réaliser une prédiction du caractère représenté sur une image totalement nouvelle.

Nota : comme en 2A, merci de lancer en parallèle le quiz Moodle du cours CCN 3A afin de répondre aux questions qui émaillent le déroulement de ce TD et pour lesquelle vous allez devoir écrire et exécuter quelques lignes de code Python.

## Principales étapes du TD

*   Importation des bibliothèques spécialisées
*   Téléchargement des images (et des étiquettes de ces images)
*   Création du modèle par ajout de couches de neurones
*   Apprentissage
*   Sauvegarde du modèle pour utilisation future
*   Evaluation du modèle
*   Utilisation du modèle pour prédiction
*   Les biais de l'apprentissage
*   Passage à l'échelle et modèles pré-entraînés
*   Un peu d'interprétabilité
*   Impact environnemental de l'IA

# Lancement du TP
## Installation des bibliothèques spécialisées

Dans ce TP nous installons utiliser les bibliothèques TensorFlow et Keras.

TensorFlow est une bibliothèque Open Source de créée par Google et compatible avec le langage Python. Il s'agit d'une boîte à outils permettant de standardiser et de simplifier la création et le déploiement de modèles de deep learning.

Keras est une bibliothèque Open Source modulaire, rapide et simple d’utilisation écrite en langage Python et exécutée par-dessus des frameworks tels que Theano et TensorFlow. Elle offre une façon simple et intuitive de créer des modèles de Deep Learning. Aujourd’hui, Keras est l’une des APIs de réseaux de neurones les plus utilisées pour le développement et le test de réseaux de neurones. Elle permet de créer très facilement un modèle en plusieurs couches, même pour des architectures complexes.

Commençons par vérifier la version de Python. C'est une information utile si les bibliothèques par défaut n'étaient pas compatibles avec cette version de Python.

In [ ]:
import sys
print("Python version:", sys.version)

##   Téléchargement des images (et des étiquettes de ces images)

Pour pour les données dentraînement, nous allons utiliser un ensemble de données déjà disponibles pour l'OCR telles que celles proposées par MNIST.
MNIST contient des images de chiffres écrits à la main.

Chargeons ces données grâce à la méthode load_data() qui charge les données dans deux tuples :
 *  le tuple (train_images, train_labels) va nous servir à entraîner le modèle ;
 *  le tuple (test_images, test_labels) va servir à estimer la précision le modèle avant son utilisation

In [ ]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

### Données d'apprentissage
Examinons ces données.  

Premières questions (n'oubliez pas de lancer le quiz Moodle en // afin de répondre aux questions qui suivent tout au long du TP) :

* Q01 : Quelle est la classe de train_images ?
* Q02 : Combien d'images contient train_images ?
* Q03 : Quelles sont les dimensions d'une image ?

Pour répondre à ces questions, vous allez devoir écrire et exécuter quelques lignes de code Python.



In [ ]:
# votre code ci-dessous


Chaque image est un tableau de w lignes et h colonnes contenant un valeur comprise entre 0 et 255.

* Q04 : Que représente selon vous les valeurs contenues dans le tableau correspondant à une image ? (3 mots)

Pour afficher une image, on utilise la librairie matplotlib, comme ci-dessous avec la première image du tableau train_images.

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(train_images[0], cmap='gray')
plt.show()


*   Q05 : quel chiffre représente l'image train_image[26]

In [ ]:
# votre code ci-dessous


Le code suivant nous affiche quelques exemples de façon plus concise.

In [ ]:
plt.figure(figsize=(10,5))
for i in range(5):
  plt.subplot(1, 5, i+1)
  plt.imshow(train_images[i], cmap='gray')
  plt.axis('off')
  plt.title(train_labels[i]);

Examinons à présent les labels.

* Q06 : Quelle est la classe de train_labels ?
* Q07 : Combien de labels contient train_labels ?
* Q08 : Quel est le label train_labels[20457] ? Vérifiez que ce label correspond bien à l'image train_image[20457]

In [ ]:
# votre code ci-dessous



### Données de test

* Q09 : Combien d'images contient test_images ?



In [ ]:
# votre code ci-dessous


# Préparation des données
Nous allons maintenant préparer les données pour les données d'entraînement et de test.  
En informatique, les images sont représentées sous forme de matrices dont les lignes et colonnes correspondent aux pixels de l'image.  
Les valeurs des pixels sont des valeurs numériques qui représentent ici le niveau de gris de chaque pixel. Ces valeurs sont comprises entre 0 et 255.

Il est plus facile de traiter d'optimiser un réseau de neurones lorsque les données sont normalisées. Nous allons donc normaliser les données d'images en les divisant par 255. De manière similaire à ce que nous avions fait pour le TD 2A, nous allons aussi transformer les données de chaque image (au départ un tableau 2D) en un tableau 1D.

Q10 : Quelle est la dimension de ce tableau 1D ?

In [ ]:
train_images = train_images.reshape((60000, 28*28)) / 255
test_images = test_images.reshape((10000, 28*28)) / 255
print(train_images.shape)

## Construction du modèle de réseau de neurones

Nous allons à présent construire un modèle de RN simple avec Keras.   
Le problème qui nous intéresse est en réalité assez simple et ne nécessite pas un gros réseau pour obtenir de bons résultats. Nous allons construire un premier réseau "fully-connected" (entre deux couches, tous les neurones sont connectés) qui prendra en entrée une image sous forme de vecteur et retournera une classification.  
![](https://img.favpng.com/13/22/3/mnist-database-multilayer-perceptron-artificial-neural-network-statistical-classification-machine-learning-png-favpng-tDc3Ze2RegCutriyH12TfquqE.jpg)

* Q11 : De combien de neurones avons nous besoin dans la couche d'entrée ?  
* Q12 : De combien de neurones avons nous besoin dans la couche de sortie ?

En plus des couches d'entrée et de sortie, notre réseau de neurones aura deux couches cachées, de respectivement 128 et 64 neurones.

On appelle fonction d'activation la façon dont un neurone transmet une information à la couche suivante. Dans le TD 2A, l'information passée à chaque neurone de sortie était simplement la combinaison linéaire des neurones reliés à cette sortie pondérée par le poids de chaque neurone.   Ici, nous allons utiliser une fonctions non-linéaire : la fonction d'activation ReLU (à vous de chercher sur Internet).

* Q13 : Que vaut ReLU(3) ?
* Q14 : Que vaut ReLU(-3) ?

Pour la dernière couche, nous allons utiliser la fonction d'activation softmax afin de produire une distribution de probabilités sur les différentes classes possibles en sortie du réseau.

[softmax](https://fr.wikipedia.org/wiki/Fonction_softmax)

La façon la plus simple de construire un réseau sous Keras est d'utiliser les models Sequential.
Il suffit d'instancier un de ces models et d'y ajouter les couches dont on a besoin une par une.

In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential()
# On commence par ajouter une couche d'entrée
model.add(layers.Input(shape=(784,)))
# On ajoute une première couche cachée de 128 neurones avec fonction d'acctivation ReLU
model.add(layers.Dense(128, activation='relu'))
# On ajoute une seconde couche cachée de 64 neurones avec fonction d'acctivation ReLU
model.add(layers.Dense(64, activation='relu'))
# On ajoute une couche de sortie avec 10 neurones (une pour chaque classe) et fonction d'activation softmax
model.add(layers.Dense(10, activation='softmax'))

* Q15 : Quel est le nombre total de paramètres à optimiser ?

Nous pouvons visualiser ce modèle :





In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='mlp_model.png')

## Entraînement du modèle

Nous allons maintenant compiler le modèle/réseau. Compiler le modèle consiste à choisir
* l'optimiseur (la méthode utilisée pour optimiser ce modèle),
* la fonction de perte (la fonction qui quantifie l'écart entre les prévisions du modèle et les observations réelles du jeu de données utilisé pendant l'entraînement - la phase d’entraînement vise à trouver les paramètres du modèle qui permettront de minimiser cette fonction),
* les métriques que nous souhaitons suivre (pour évaluer le pourcentage de prédictions correctes).  

Dans notre cas, nous allons utiliser
* l'optimiseur Adam,
* la fonction de perte sparse_categorical_crossentropy
* la métrique accuracy.

Nous n'entrerons pas ici dans les détails de ce choix qui dépasse le cadre de ce module. Notez seulement que Sparse_categorical_crossentropy est la fonction de perte appropriée lorsque nous avons un problème de classification où les étiquettes sont données sous forme de numéros entiers.


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Phase d'apprentissage

Deux éléments sont essentiels, ici :
* la taille du batch
* le nombre d'époques

La taille du batch fait référence au nombre d'échantillons d'entraînement utilisés pour cette phase. Cette taille influence à la fois la vitesse d'apprentissage et la précision des mises à jour de poids. Une taille de batch plus petite peut augmenter la précision des mises à jour, tandis qu'une taille de batch plus grande peut accélérer le processus d'apprentissage. En général, une taille de batch de 32 ou 64 est recommandée comme point de départ.

Le nombre d'époques désigne le nombre de passages qu’un jeu de données d’entraînement effectue autour d’un algorithme. Une époque est terminée lorsque tous les échantillons de données ont été exposés au réseau de neurones. Ce processus se répète jusqu’à ce que son taux d’erreur soit satisfaisant. Le nombre d’Epochs utilisé influe sur la vitesse et la qualité de l'apprentissage. Lorsque le nombre d’époques est trop faible, le réseau de neurones ne sera pas suffisamment entraîné et ses performances seront médiocres. Lorsque le nombre d’Epochs est trop élevé, le réseau neuronal risque de faire du surajustement ou overfitting. Il sera alors tout aussi incapable de réaliser des prédictions.

In [ ]:
# model.fit(train_images, train_labels, epochs=5, batch_size=64)
batch_size = 128
epochs = 10

history = model.fit(train_images, train_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(test_images, test_labels))


### Sauvegarde du modèle

Dans cette phase, nous sauvegardons le modèle entraîné. Ainsi, nous ne serons pas obligés d'exécuter toutes les actions précédentes à chaque nouvelle utilisation. Keras fournit un format de sauvegarde de base utilisant la norme HDF5, un format spécialement conçu pour sauvegarder de grandes masses de données.

In [ ]:
model.save('model.h5')

Vous pouvez télécharger ce modèle (depuis l'espace de fichiers) et l'exécuter dans un autre environnement (par exemple localement au sein d'un notebook Python).
Un modèle Keras contient les composants suivants :
The architecture, or configuration, which specifies what

*   l'architecture du modèle (les différentes couches et leur interconnexion)
*   les poids des différents neurones
*   un optimiseur (défini au moment de la compilation)
*   les métriques du modèle en termes de précision


## Évaluation du modèle

Dans cette partie, nous évaluons la précision du modèle sur les données de test.
Dans le code ci-dessous, la variable score est un tableau contenant deux variables :
* la valeur de la fonction de perte
* la précision du modèle (accuracy)


In [ ]:
score = model.evaluate(test_images, test_labels, verbose=0)
print(score)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Q16 : Quelle est la valeur de la fonction de perte à 0,05 près ?

Q17 : Quelle est la précision du modèle à 0,05 près ?

On peut aussi tracer les courbes d'apprentissage afin de visualiser l'évolution de la précision et de la perte au cours des époques.  

In [ ]:
import matplotlib.pyplot as plt

def plot_learning_curves(history):
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()


In [ ]:
# rappel : history est le résultat de la phase d'apprentissage
plot_learning_curves(history)

Vous drevriez avoir quelque chose comme 85% d'accuracy sur les données de test.  
On peut faire beaucoup mieux avec un réseau de neurones plus profond ou avec une architecture différente.

Travail à effectuer : ajouter une couche cachée de 64 neurones pour voir si cela améliore les performances
(montrer à l'intervenant)

Optionnel : vous pouvez aussi augmenter la taille des couches cachées si ça ne prend pas trop de temps.  

In [ ]:
from tensorflow.keras.optimizers import Adam

model = models.Sequential()
model.add(layers.Input(shape=(784,)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer = Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(test_images, test_labels))

score = model.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Prédiction

Dans cette partie, nous allons utiliser le modèle afin de faire des prédictions sur des images nouvelles.

**(il faut recompiler ? avec un autre optimizer ?)**

In [ ]:
# charger le modèle
from keras.models import load_model
import numpy as np

model = load_model('model.h5')
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Prediction sur une image de test_images

L'instruction

```
f = model.predict(np.expand_dims(test_images[1567], axis=0))
```

permet de faire une prédiction sur l'image 1567 du jeu de test : f[0] contient les valeurs des 10 sorties du réseau.

Afficher le résultat de cette prédiction.

* Q18 : Quel chiffre le modèle prédit-il sur cette image ?
* Q19 : Est-ce cohérent ?

In [ ]:
# votre code ici


Complétez la fonction ci-dessous pour qu'elle affiche la prédiction du modèle sur une serie d'images.

In [ ]:
plt.figure(figsize=(10,5))
# votre code ici
for i in range(5):
...

# Généralisation et biais
Nous allons maintenant tester notre modèle sur de nouvelles images.  
Pour commencer, nous allons utiliser un outil appelé Gradio. Gradio est un package Python open source qui permet de créer rapidement des composants d'interface utilisateur personnalisables et faciles à utiliser. Ici, gradio va nous permettre de dessiner des chiffres grâce à un sketchpad.

In [ ]:
!pip install gradio > /dev/null

En cas d'erreur sur cette installation, insérer le # de commentaire avant "> /dev/nul"

### Prediction sur une image nouvelle

Exécuter le code ci-dessous qui configure et lance le sketchpadm notq;;ent en définissant la fonction qui traite toute nouvelle image : charge de l'image dans Python, reformatage de l'image pour qu'elle soit en N&B et de taille 28x28 et lance la prédiction sur cette image.

Dès que le sketchpad s'affiche, dessiner un chiffre et observer la prédiction faite.

In [ ]:
import gradio as gr
import numpy as np
import tensorflow as tf

def classify(image):
    # Récupération de l'image depuis le sketchpad
    image = image["composite"]

    # Conversion de l'image en numpy array et niveaux de gris
    image = np.array(image)
    if len(image.shape) == 3:
        image = np.mean(image, axis=2)

    # Redimensionnement
    image = tf.keras.preprocessing.image.smart_resize(
        np.expand_dims(image, -1), [28, 28]
    )
    print(image.shape)

    # Normalisation et reshape
    image = (image / 255.0).reshape(1, 28 * 28)

    # Prédiction
    prediction = model.predict(image, verbose=0).tolist()[0]
    return {str(i): prediction[i] for i in range(10)}

# Utilisation d'un sketchpad
sketchpad = gr.Sketchpad()
label = gr.Label(num_top_classes=3)
interface = gr.Interface(classify, sketchpad, label, live=True)

interface.launch(debug=True)

Vous devriez obtenir des resultats sensiblement moins bons que sur le jeu de test.  
Cela vient du fait que les données utilisées pour le test sont très proches de celle que le modèle a appris. Ce n'est pas le cas des images que vous êtes en train de dessiner.  
Les réseaux de neurones sont entraînés à prédire sur une distribution de données qui est très proche de celle qu'ils ont apprise.  Lorsque l'on s'en éloigne, les performances du modèle baissent.  

Parfois il est difficile pour un humain de se rendre compte que la donnée suit une distribution différente. Pour illustrer ceci, nous allons créer un nouveau jeu de test en appliquant une très légère translation à chaque image du jeu de test. Après avoir exécuté l'exemple simple ci-dessous, écrivez la ligne qui permet d'appliquer une translation horizontale de 5 pixels.

* Q20 : Au fait, que fait exactement la fonction roll ?

In [ ]:
# exemple d'utilisation de roll
import numpy as np
x = np.arange(10)
print(x)
# application de la méthode roll
x = np.roll(x, 2)
print(x)

In [ ]:
# création d'un nouveau jeu par copie
shifted_test_images = test_images.copy()
# translation (à vous d'écrire la ligne)
shifted_test_images = np.roll(shifted_test_images, 5)
# affichage
plt.figure(figsize=(10,5))
plt.figtext(0.5, 0.9, 'Original TestSet', ha='center', fontsize=12)
plt.figtext(0.5, 0.45, 'Shifted TestSet', ha='center', fontsize=12)
for i in range(5):
  plt.subplot(2, 5, i+1)
  plt.imshow(test_images[i].reshape(28, 28), cmap='gray')
  plt.axis('off')
for i in range(5):
  plt.subplot(2, 5, i+6)
  plt.imshow(shifted_test_images[i].reshape(28, 28), cmap='gray')
  plt.axis('off')


A l'oeil nu, on ne voit pas de grande différence et cela reste compréhensible pour un humain.  
Cependant lorsque l'on regarde la distribution des valeurs maximales en x, entre ce dataset et le précédent on observe une différence significative.  

In [ ]:
import seaborn as sns
# plot the distribution of max in x and y axis using seaborn histograms
plt.figure(figsize=(8,5))

# Calculate max positions along x and y axes
x_max_pos_orig = np.argmax(np.mean(test_images.reshape(10000, 28, 28, 1).squeeze(), axis=1), axis=1)
y_max_pos_orig = np.argmax(np.mean(test_images.reshape(10000, 28, 28, 1).squeeze(), axis=2), axis=1)
x_max_pos_shift = np.argmax(np.mean(shifted_test_images.reshape(10000, 28, 28, 1).squeeze(), axis=1), axis=1)
y_max_pos_shift = np.argmax(np.mean(shifted_test_images.reshape(10000, 28, 28, 1).squeeze(), axis=2), axis=1)

# Plot distributions
sns.histplot(data={"Original": x_max_pos_orig, "Shifted": x_max_pos_shift}, multiple="layer", bins=28)
plt.title("Distribution of max positions (X-axis)")
plt.xlabel("Position")
plt.ylabel("Count")

plt.tight_layout()


Comparons les résultats de la prédiction sur le jeu de test original et sur le jeu de test décalé:

In [ ]:
score = model.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

shifted_test_images = test_images.copy()
shifted_test_images = np.roll(shifted_test_images, 5, axis=1)

score = model.evaluate(shifted_test_images, test_labels, verbose=0)
print('Shifted Test loss:', score[0])
print('Shifted Test accuracy:', score[1])

In [ ]:
plt.figure(figsize=(10,5))
plt.figtext(0.5, 0.9, 'Prediction on Original TestSet', ha='center', fontsize=12)
plt.figtext(0.5, 0.45, 'Prediction on Shifted TestSet', ha='center', fontsize=12)
for i in range(5):
  plt.subplot(2, 5, i+1)
  plt.imshow(test_images[i].reshape(28, 28), cmap='gray')
  plt.axis('off')
  label = model.predict(np.expand_dims(test_images[i], axis=0))
  label = np.argmax(label)
  plt.title(label);

for i in range(5):
  plt.subplot(2, 5, i+6)
  plt.imshow(shifted_test_images[i].reshape(28, 28), cmap='gray')
  plt.axis('off')
  label = model.predict(np.expand_dims(shifted_test_images[i], axis=0))
  label = np.argmax(label)
  plt.title(label);

Un peu triste n'est-ce pas ?  Nous venons de montrer ici que les réseaux de neurones sont très sensibles aux petites variations dans les données.  
En réalité les réseaux de neurones ne généralisent pas si bien que l'on pourrait le croire et apprènent les biais présents dans les données d'apprentissage.  
Dans le cas de notre exemple, le modèle a appris à prédire les chiffres de manière incorrecte en se basant sur aussi bien la position du chiffre dans l'image que sur la forme du chiffre. Un simple décalage de 5 pixels dans l'image suffit à le faire échouer complètement.  
Les biais présents dans les données d'apprentissage sont souvent difficiles à identifier pour un humain et peuvent avoir des effets dramatiques sur les performances du modèle ou entraîner des biais dans les prédictions.  [Un exemple assez connu est le biais de genre dans les données d'apprentissage.](https://www.reuters.com/article/world/insight-amazon-scraps-secret-ai-recruiting-tool-that-showed-bias-against-women-idUSKCN1MK0AG/)  
Soyez donc toujours vigilant sur les données que vous utilisez pour entraîner vos modèles et essayez de les tester sur des données qui ont été générées dans des conditions différentes pour avoir une meilleure idée de la généralisation du modèle.  

# Impact environnemental de l'IA

Vous savez sans doute que le secteur du numérique représente environ 4% des émissions à effet de serre. Il pourrait en représenter 8% en 2050.

La seule mise au point des modèles d’apprentissage profond qui sont utilisés massivement aujourd’hui requiert l’optimisation de millions, voire de milliards de paramètres et nécessite donc beaucoup de temps de calcul et d’énergie. Un des premiers travaux sur cette question du coût énergétique de l’Intelligence Artificielle est paru en 2020. Il a marqué les esprits en montrant que la mise au point de certains modèles pouvait représenter jusqu’à 300 tonnes équivalent CO2. 300 tonnes équivalent CO2 correspondent à environ 300 aller-retours transatlantiques en avion.    

Aujourd’hui, il existe de nombreuses études et méthodologies de suivi de la consommation énergétique pour l’IA qui prennent en compte le cycle de vie complet de l’IA.

Dans cette partie, nous allons juste tenter d'avoir une estimation du coût carbone lié à la mise au point de notre modèle. Pour ce faire, nous allons utiliser le package Python Codecarbon.

Commençons par installer le package. Revenez au début de la phase d'apprentissage, ajoutez et exécuter une cellule avec le code suivant :


In [ ]:
!pip install jedi==0.16.0
!pip install codecarbon
!mkdir codecarbon

Puis définissez un tableau pour recevoir les résultats:

In [ ]:
tableau = []

Le tracker Codecarbon est lancé avant l'exécution du model.fit() et stoppé juste après. A vous de modifier la cellule contenant le model.fit() selon ce qui vous est donné ci-dessous.


In [ ]:
# lancement du tracker
from codecarbon import EmissionsTracker
tracker = EmissionsTracker(
    output_dir="./codecarbon/",  # dossier pour les résultats
    output_file="emissions.csv",  # fichier contenant les résultats
)
tracker.start()

# model.fit(train_images, train_labels, epochs=5, batch_size=64)
# ...

# tracker stop
emissions: float = tracker.stop()
print(f"Emissions pour {epochs} époque(s) : {emissions:8f} kg CO2eq")
resultats.append([epochs,emissions])


Enfin le code ci-dessous permet d'afficher les émissions CO2 en fonction du nombre d'époques.

> Add blockquote



In [ ]:
# reformatage des résultats (deux tableaux)
emi=[]
epo=[]
for r in resultats:
  epo.append(r[0])
  emi.append(r[1])

# fonction d'affichage des résultats
def plot_emission(epo,emi):
  plt.plot(epo,emi)
  plt.title('Emissions selons nombre d\'époques')
  plt.ylabel('Emissions')
  plt.xlabel('Nombre d\'époques')
  plt.show()

plot_emission(epo,emi)

A vous donc de modifier plus haut la cellule contenant le model.fit pour

* lancer le tracker
* lancer le model.fit
* arrêter le tracker

Cette cellule devra être exécutée plusieurs fois afin d'enrichir le tableau des résultats.

# Conclusion
Ce notebook fournit une vue d'ensemble pour vous aider à comprendre le fonctionnement de base des réseaux de neurones dans le contexte de la reconnaissance de caractères. Dans ce TD, vous avez

* utilisé des bibliothèques spécialisées pour construire et configurer un réseau de neurones contenant plusieurs couches
* entraîné et testé ce réseau grâce à de grandes bibliothèques d'images
* évalué sa précision,
* utilisé ce modèle pour réaliser une prédiction du caractère représenté sur une image totalement nouvelle

Ce modèle est bien largement perfectible.

Nous espérons quer ce Notebook vous a donné un aperçu des principes et enjeux techniques autour des réseaux de neurones.

D. Bertoin et J.-Y. Plantec, 01/2025
